In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from keras.models import Sequential
from keras.layers import Activation
from keras.optimizers import SGD
from keras.layers import Dense
from keras.utils import np_utils
import numpy as np
import itertools
import tables

In [5]:
pulses_file = "./data/inicedstpulses_nugen11069_first50i3files.h5"

take care with that dataset: entry (1106900050, 1612) appeares to be available twice in MCPrimary table... delete it manually.

In [6]:
def load_geo_data(geo_file):
    #return the contents of the geo file as
    #dictionary mapping (omkey[0],omkey[1]) -> (posx,posy,poz)
    import csv
    ret = {}
    with open(geo_file, 'r') as f:
        reader = csv.reader(f)
        for line in reader:
            key = tuple(map(int, line[0:2]))
            val = tuple(map(float, line[2:5]))
            ret[key] = val
    return ret

In [7]:
def produce_raw_xlist(geo=None):
    """
    returns a two tuple:
    1) a np array of lenght #DOMS filled with zeroes. copy this for every dataset later on
    2) a dictionary mapping (string, om) to an index (something like $index = (string-1)*60+(om-1)$)
    
    geo is the object created by load_geo_data. 
    if no geo specified use standard values: string in [1,86], om in [1,60]
    """
    if geo:
        dom_to_index = {}
        raw_xlist = np.zeros((len(geo)), dtype=np.float32)
        for i, dom in enumerate(geo.keys()):
            raw_xlist[i] = 0.0 #this will also be the value used for non hit DOM's
            dom_to_index[dom] = i
        return raw_xlist, dom_to_index
    else: #no geo file specified, use standard values: string in [1,86], om in [1,60]
        dom_to_index = {}
        for i, (string, om) in enumerate(itertools.product(range(1,87), range(1,61))):
            dom_to_index[(string, om)] = i
        raw_xlist = np.zeros((86*60), dtype=np.float32)
        return raw_xlist, dom_to_index

In [21]:
def load_pulses_data(pulses_file, geo_file=None):
    #returns data from dstpulses. Returns only the first puls from each dom in the list.
    #splits per frames
    """Return a tuple containing ``(data,  labels)``.
    
    In particular, ``data`` is a list containing a dozen thousand
    lists ``[chargedom1,chargedom2,...]``.  the index of the dom is simply a walkthrough through 
    every (string, om)-pair
    
    ``labels`` is the
    corresponding information about being a north or south-coming particle,
    i.e., 0 for above (south) and 1 for below (north). this is chosen on the
    zenith angle of MCTree's most energetic primary (1 for > or 0 for < 90deg). it is made categorical:
    [[0,1],[0,1],[1,0],...] for [down, down, up, ...]

    For DOMs that include multiple pulses, -the earliest time is used for simplification- sum of charge is used.
    """

    def get_pos(dom_to_index, string, om):
        if (string, om) not in dom_to_index:
            return -1
        return dom_to_index[(string, om)]

    def normalize_time(time_list):
        maxtime, mintime = 0, float("inf")
        for t in time_list[:,0]:
            maxtime = max(maxtime, t)
            mintime = min(mintime, t)
        timespan = maxtime - mintime
        time_list[:,0] -= mintime
        time_list[:,0] /= timespan
    def normalize_charge(charge_list):
        maxcharge, mincharge = 0, float("inf")
        for c in charge_list:
            maxcharge = max(maxcharge, c)
            mincharge = min(mincharge, c)
        span = maxcharge - mincharge
        charge_list -= mincharge
        charge_list /= span

    raw_xlist, dom_to_index = produce_raw_xlist()

    h5file = tables.open_file(pulses_file)
    dst = h5file.root.InIceDSTPulses.cols
    prim = h5file.root.MCPrimary.cols

    data = []   #total charge (summed over pulses) per dom and per frame. 2d numpy array
    labels = [] #up or down, categorical [[0,1],[0,1],[1,0],...]. 2d numpy array
    prev_frame = (dst.Run[0],dst.Event[0])
    prev_dom = (-1,-1)
    x_list = np.copy(raw_xlist)
    count = 0
    x_i = 0

    for zenith in prim.zenith:
        labels.append(1 if zenith > 1.5707963268 else 0) #1==down, 0==up
    labels = np_utils.to_categorical(labels)

    total_rows = len(dst.Run)
    i=0
    for run, event, string, om, time, charge in zip(dst.Run, dst.Event, dst.string, dst.om, \
                                                    dst.time, dst.charge):
        if (run, event) != prev_frame: #next frame is coming, so push this out as charges list
            normalize_charge(x_list)
            data.append(x_list)
            x_list = np.copy(raw_xlist)
            count += 1
            prev_frame = (run,event)

        if (string, om) == prev_dom: #already saw that dom (it has multiple pulses)
            x_list[dom_index] += charge
        else: #pulse for new dom
            dom_index = get_pos(dom_to_index, string, om)
            if dom_index == -1: #this must be one of those om=61,62,63,64 (i.e. icetop). we're not interested in them
                continue
            x_list[dom_index] = charge
            prev_dom = (string, om)
            
        # show an update every 1,000 images
        if i > 0 and i % 10**6 == 0:
            print("[INFO] processed {}/{}".format(i, total_rows))
        i += 1

    #add the last frame manually
    normalize_charge(x_list)
    data.append(x_list)

    return (np.array(data), labels)

In [22]:
data, labels = load_pulses_data(pulses_file)

[INFO] processed 1000000/43287699
[INFO] processed 2000000/43287699
[INFO] processed 3000000/43287699
[INFO] processed 4000000/43287699
[INFO] processed 5000000/43287699
[INFO] processed 6000000/43287699
[INFO] processed 7000000/43287699
[INFO] processed 8000000/43287699
[INFO] processed 9000000/43287699
[INFO] processed 10000000/43287699
[INFO] processed 11000000/43287699
[INFO] processed 12000000/43287699
[INFO] processed 13000000/43287699
[INFO] processed 14000000/43287699
[INFO] processed 15000000/43287699
[INFO] processed 16000000/43287699
[INFO] processed 17000000/43287699
[INFO] processed 18000000/43287699
[INFO] processed 19000000/43287699
[INFO] processed 20000000/43287699
[INFO] processed 21000000/43287699
[INFO] processed 22000000/43287699
[INFO] processed 23000000/43287699
[INFO] processed 24000000/43287699
[INFO] processed 25000000/43287699
[INFO] processed 26000000/43287699
[INFO] processed 27000000/43287699
[INFO] processed 28000000/43287699
[INFO] processed 29000000/432

In [75]:
h5=tables.open_file(pulses_file)
mcp=h5.root.MCPrimary.cols
index_to_remove = -1
for i, (run,event) in enumerate(zip(mcp.Run, mcp.Event)):
    if (run, event) == (1106900050, 1612):
        index_to_remove = i
        break
if len(labels) > len(data): #just to be sure...
    labels = np.delete(labels, index_to_remove, 0)

In [77]:
(trainData, testData, trainLabels, testLabels) = train_test_split(data, labels, test_size=0.25, random_state=42)

In [78]:
model = Sequential()
model.add(Dense(1024, input_dim=5160, kernel_initializer="uniform",
	activation="relu"))
model.add(Dense(512, kernel_initializer="uniform", activation="relu"))
model.add(Dense(2))
model.add(Activation("softmax"))

In [79]:
#train the model
sgd = SGD(lr=0.01)
model.compile(loss="binary_crossentropy", optimizer=sgd,
	metrics=["accuracy"])
model.fit(trainData, trainLabels, epochs=20, batch_size=128)

Epoch 1/50
7875/7875 [==============================] - 3s - loss: 0.6918 - acc: 0.5342     
Epoch 2/50
7875/7875 [==============================] - 3s - loss: 0.6902 - acc: 0.5458     
Epoch 3/50
7875/7875 [==============================] - 3s - loss: 0.6894 - acc: 0.5450     
Epoch 4/50
7875/7875 [==============================] - 3s - loss: 0.6889 - acc: 0.5450     
Epoch 5/50
7875/7875 [==============================] - 3s - loss: 0.6884 - acc: 0.5453     
Epoch 6/50
7875/7875 [==============================] - 3s - loss: 0.6879 - acc: 0.5450     
Epoch 7/50
7875/7875 [==============================] - 3s - loss: 0.6875 - acc: 0.5453     
Epoch 8/50
7875/7875 [==============================] - 3s - loss: 0.6870 - acc: 0.5464     
Epoch 9/50
7875/7875 [==============================] - 3s - loss: 0.6865 - acc: 0.5468     
Epoch 10/50
7875/7875 [==============================] - 4s - loss: 0.6861 - acc: 0.5468     
Epoch 11/50
7875/7875 [==============================] - 3s - loss: 0

In [80]:
# show the accuracy on the testing set
print("[INFO] evaluating on testing set...")
(loss, accuracy) = model.evaluate(testData, testLabels,
	batch_size=128, verbose=1)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,
	accuracy * 100))

[INFO] evaluating on testing set...
2432/2626 [==========================>...] - ETA: 0s[INFO] loss=0.6787, accuracy: 56.5499%


# results #
feedforward neural net 5160-1024-512-2, relu activations. trained on 25%